모수검정 中 분산분석(ANOVA)
: 다집단 모평균 검정

- 두 집단 이상의 모평균을 검정
- 실험 설계, 반응변수, 요인의 수에 따라 다양하게 구분 가능
- 일원배치 분산분석
- 이원배치 분산분석
- 사후 검정

In [14]:
# A~C 3개 학교에서 각 1000명씩 층화 추출한 데이터를 활용
# 세 학교 간 기말고사 성적의 평균이 서로 차이가 있는지 검정
# 성별, 학년의 비율은 세 학교가 동일
# 정규성, 등분산성 등 만족 가정

# => 정규성, 등분산성 만족하기에 모수 검정
# => 모평균 검정인데, 다표본이기에 ANOVA

import pandas as pd

# anova 분석을 위해서는 ols(ordinary least squares)
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

df = pd.read_csv('C:/BigData/practice_data/예제/school_score.csv')
#print(df.columns)

one_way = ols('Final ~ C(School)', data=df).fit()
result = anova_lm(one_way)
print(result)

'''
결과
df         sum_sq        mean_sq            F  PR(>F)
C(School)     2.0  996939.237147  498469.618573  5722.221007     0.0
Residual   2997.0  261072.308290      87.111214          NaN     NaN
'''
# School 기준 그룹들의 p-value 작음 -> 귀무가설 기각 -> 차이 존재.

# 결론: 세 학교 간 기말고사 성적의 평균이 서로 차이가 존재!


               df         sum_sq        mean_sq            F  PR(>F)
C(School)     2.0  996939.237147  498469.618573  5722.221007     0.0
Residual   2997.0  261072.308290      87.111214          NaN     NaN


'\n결과\ndf         sum_sq        mean_sq            F  PR(>F)\nC(School)     2.0  996939.237147  498469.618573  5722.221007     0.0\nResidual   2997.0  261072.308290      87.111214          NaN     NaN\n'

In [23]:
# 위에서 사용한 데이터를 활용해
# 학교와 성별이 기말고사 성적에 영향을 주는지 알아보기 위해
# 이원배치 분산분석을 수행

# => 학교 AND 성별 -> 교호작용 필요
# => 교호작용 포함한 이원배치 분산 분석!
# => 분산 분석에는 ols 객체 생성 선행 필요

import pandas as pd
df = pd.read_csv('C:/BigData/practice_data/예제/school_score.csv')
# print(df.columns)

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

two_way = ols('Final ~ C(School) + C(Sex) + C(School):C(Sex)', data = df).fit()
result = anova_lm(two_way)
print(result)

'''
결과
                      df         sum_sq        mean_sq            F    PR(>F)
C(School)            2.0  996939.237147  498469.618573  5720.301475  0.000000
C(Sex)               1.0      30.257511      30.257511     0.347227  0.555732
C(School):C(Sex)     2.0     143.557155      71.778578     0.823711  0.438899
Residual          2994.0  260898.493624      87.140445          NaN       NaN
'''
# School 기준 그룹들의 p-value 작음 -> 귀무가설 기각 -> 차이 존재
# Sex 기준 그룹들의 p-value 큼 -> 귀무가설 채택 -> 차이 없음
# School + Sex (교호작용) 그룹들의 p-value 큼 -> 귀무가설 채택 -> 차이 없음

# 결론: 상호작용 효과가 통계적으로 유의하지 않음! -> 교호작용 고려하지 않고 다시 분산 분석 수행

two_way = ols('Final ~ C(School) + C(Grade)', data=df).fit()
result = anova_lm(two_way)
print(result)
'''
결과
               df         sum_sq        mean_sq            F    PR(>F)
C(School)     2.0  996939.237147  498469.618573  5725.518315  0.000000
C(Grade)      2.0     324.472829     162.236415     1.863479  0.155312
Residual   2995.0  260747.835461      87.061047          NaN       NaN
'''
# Grade 기준 그룹들의 p-value 큼 -> 귀무가설 채택 -> 차이 없음

# 결론: 유의수준 5%에서 학년에 따른 기말 고사 평균은 유의한 차이가 없음!

                      df         sum_sq        mean_sq            F    PR(>F)
C(School)            2.0  996939.237147  498469.618573  5720.301475  0.000000
C(Sex)               1.0      30.257511      30.257511     0.347227  0.555732
C(School):C(Sex)     2.0     143.557155      71.778578     0.823711  0.438899
Residual          2994.0  260898.493624      87.140445          NaN       NaN
               df         sum_sq        mean_sq            F    PR(>F)
C(School)     2.0  996939.237147  498469.618573  5725.518315  0.000000
C(Grade)      2.0     324.472829     162.236415     1.863479  0.155312
Residual   2995.0  260747.835461      87.061047          NaN       NaN


'\n결과\n               df         sum_sq        mean_sq            F    PR(>F)\nC(School)     2.0  996939.237147  498469.618573  5725.518315  0.000000\nC(Grade)      2.0     324.472829     162.236415     1.863479  0.155312\nResidual   2995.0  260747.835461      87.061047          NaN       NaN\n'

In [30]:
# 위에서 수행된 분산분석 이후 과정인 사후 검정을 수행
# 튜키의 다중비교 진행

import pandas as pd
df = pd.read_csv('C:/BigData/practice_data/예제/school_score.csv')

from statsmodels.stats.multicomp import pairwise_tukeyhsd
tukey_result = pairwise_tukeyhsd(endog=df.Final, groups = df.School, alpha = 0.05)
print(tukey_result)

'''
결과
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
====================================================
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     A      B  38.8464   -0.0 37.8677 39.8251   True
     A      C  38.4922   -0.0 37.5135 39.4709   True
     B      C  -0.3542 0.6728 -1.3329  0.6245  False
----------------------------------------------------
'''
# 모든 쌍에 대해 분산 분석 -> 어느 쌍에서 차이가 발생하는지 확인 가능 (사후검정)
# B,C 쌍에서 lower, upper 사이 구간에 0 포함 -> 차이 없음
# 나머지 쌍에서 lower, upper 사이 구간에 0 포함 X -> 차이 존재!

# 결론: B와 C는 신뢰구간에 0을 포함하므로 B와 C 학교 간 기말고사 평균에는 차이가 없다!

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     A      B  38.8464   -0.0 37.8677 39.8251   True
     A      C  38.4922   -0.0 37.5135 39.4709   True
     B      C  -0.3542 0.6728 -1.3329  0.6245  False
----------------------------------------------------


'\n결과\nMultiple Comparison of Means - Tukey HSD, FWER=0.05 \n====================================================\ngroup1 group2 meandiff p-adj   lower   upper  reject\n----------------------------------------------------\n     A      B  38.8464   -0.0 37.8677 39.8251   True\n     A      C  38.4922   -0.0 37.5135 39.4709   True\n     B      C  -0.3542 0.6728 -1.3329  0.6245  False\n----------------------------------------------------\n'